# Clasificación de Sentimientos Utilizando Regresión Lógistica

Realizado por:
- Paula Monge
- Emmanuel Zúñiga

Trabajo de Invetigación del Curso:
- Investigación en Ciencias de la Computación

Tema de Investigación:
- Aprendizaje Automático en la clasificación de Sentimientos en Twitter: Comparación de los Algoritmos de LSTM Bidireccional, BERT y Naïve Bayes

## Importación de librerias 📚

In [ ]:

!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score as acc
from sklearn.pipeline import Pipeline
from wordcloud import WordCloud
import seaborn as sns

## Carga del Dataset

In [ ]:
tweets = pd.read_csv('https://raw.githubusercontent.com/Emmazch22/Paula_y_Emmanuel_CI0134/main/Datasets/new_dataset_balanced.csv')

Observamos los primeros 10 registros del dataset

In [ ]:
tweets.head(10)

,Unnamed: 0,Text,Label
0,36205,es lo menos,0
1,105993,jaja me acoste a eso de las 3 de la manana o ...,0
2,77438,manana ire solari no mas,0
3,93306,tomate un acetaminofen amor,0
4,83950,lo siento,0
5,61508,o sea iba a quedar feo como mujer,0
6,59726,ademas de tener insomnio tengo hambre,0
7,39053,a mi me salio una en ese mismo lugar,0
8,49669,que alguien me hablee lt 3 lt 3 me aburro...,0
9,83894,ni me daras nada,0


In [ ]:
tweets.shape

(110720, 3)

In [ ]:
tweets = tweets.dropna()
tweets.shape

(110716, 3)

## Creación del Modelo de Regresión Lógistica 📈


In [ ]:
tweets_shuffled = tweets.sample(frac= 1, random_state=0)
x = tweets_shuffled["Text"]
y = tweets_shuffled["Label"]

size = len(x)

Para la distribución de los datos, se escoge una proporción de 80:20, 80% de los datos son para entrenar el modelo, y el restante para probarlo.

In [ ]:
model = Pipeline((("vect", TfidfVectorizer()),("clc", LogisticRegression(max_iter=1000))))
xTrain = x.head(int(size * 0.8))
xTest =  x.tail(int(size * 0.2))
yTrain = y.head(int(size* 0.8))
yTest =  y.tail(int(size * 0.2))
model.fit(xTrain,yTrain)



Pipeline(steps=[('vect', TfidfVectorizer()),
                ('clc', LogisticRegression(max_iter=1000))])

In [ ]:
Pipeline(steps=[('vect', TfidfVectorizer()),
                ('clc', LogisticRegression(max_iter=1000))])

Pipeline(steps=[('vect', TfidfVectorizer()),
                ('clc', LogisticRegression(max_iter=1000))])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.metrics import classification_report
#accuracy = (acc(model.predict(xTest), yTest))
# print("Accuracy :- ", accuracy)
# yPrediction = model.predict(xTest)
# Haz predicciones en el conjunto de prueba
y_pred = model.predict(xTest)

# Calcula las métricas de precisión, exactitud y exhaustividad
precision = precision_score(yTest, y_pred)
accuracy = accuracy_score(yTest, y_pred)
recall = recall_score(yTest, y_pred)
f1 = f1_score(yTest, y_pred)
print(classification_report(yTest, y_pred))
print(f'Precision: {precision:.2f}')
print(f'Accuracy: {accuracy:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

              precision    recall  f1-score   support

           0       0.76      0.80      0.78     11111
           1       0.79      0.75      0.77     11032

    accuracy                           0.77     22143
   macro avg       0.77      0.77      0.77     22143
weighted avg       0.77      0.77      0.77     22143

Precision: 0.79
Accuracy: 0.77
Recall: 0.75
F1 Score: 0.77


Para probar el modelo debe de ingresarse un texto en las comillas simples ubicadas dentro de los corchetes, posteriormente el modelo determinará la clase del texto introducido.

In [ ]:
print(model.predict(['Estoy triste']))

[0]
